Demonstrate the base functionality of the firmware.  With a loop-back cable installed from the DAC output to the ADC input, this produces a tone at the output, reads it back, and demodulates back to DC.

The TopSoc class chooses the firmware based on the board, ZCU111 or ZCU216.

For the ZCU111, the loopback cable is from DAC229_T1_CH2 to ADC224_T0_CH0, (single ended)

For the ZCU216, the loopback cables are from  226 CH 0 and 230 CH0.  The P and N are separate, so you need two physical cables.  ADC Vin channels P 0_226 and N 01_226 connect to a tile, LMFB DAC 02. Connect the LMFB DAC 02 tile to the LMFB DAC 01 tile on the left side of the board.  You need two cables running from that LMFB DAC 01 tile to channels 0_230 and 2_230 in the DAC Vout connector. 


In [1]:
from mkids import TopSoc
from scipy.signal import welch
from numpy.fft import fftshift
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load bitstream with custom overlay
soc = TopSoc()

In [3]:
soc.set_mixer(1000) # MHz
foutRequested = 500.3
foutRequested = soc.fsIn/2 + 123


In [4]:
# Create an output tone
def measurePhaseOriginal(foutRequested, doPlot=False, verbose=False):
    soc.dds_out.alloff()
    foutQuantized = soc.DF*np.round(foutRequested/soc.DF)
    pfb_ch, dds_freq, _, _ = soc.pfb_out.freq2ch(foutQuantized, soc.get_mixer())
    if verbose:
        print("output channel =",pfb_ch)
    print("output DDS freq =",dds_freq)
    soc.dds_out.ddscfg(ch=pfb_ch, f=dds_freq, g=0.9, fi=0.0)
    soc.pfb_out.qout(0)
    fInAliased = soc.fAliasedFromFTone(foutQuantized)
    print("foutQuantized =",foutQuantized)
    print("fInAliased =",fInAliased)
    decimation = 2
    pfbInQout = 8
    K, dds_freq, pdb_freq, ch = soc.pfb_in.freq2ch(np.array([fInAliased]))
    print("K =",K, "dds_freq =",dds_freq)
    stream, stream_idx = soc.chsel.ch2idx(K)
    print("stream, stream_idx",stream,stream_idx)
    inCh, offset = soc.inFreq2chOffset(fInAliased)
    print("inCh =",inCh, "offset =",offset)
    ntran, addr, bits = soc.chsel.ch2tran(inCh)
    num_tran, tran_idx = soc.chsel.set(K)
    print("num_tran, tran_idx",num_tran,tran_idx)
    ntranByTone, streamByTone = soc.inFreq2NtranStream(np.array([fInAliased]))
    print("ntranByTone, streamByTone",ntranByTone, streamByTone)

    soc.pfb_in.qout(pfbInQout)
    soc.ddscic.decimation(decimation)
    soc.ddscic.dds_outsel(outsel="product")
    soc.ddscic.set_ddsfreq(ch_id=inCh, f=offset)

    nt = 1
    nsamp = 10000
    packets = soc.stream.transfer(nt=nt, nsamp=nsamp)
    
    print("     packets.shape =",packets.shape)
    ntrans = packets[:,:,16]
    print("      ntrans.shape =",ntrans.shape)
    xis = packets[:,:,0:16:2]
    print("         xis.shape =",xis.shape)
    xqs = packets[:,:,1:17:2]
    print("         xqs.shape =",xqs.shape)
    xs = xis + 1j*xqs
    print("          xs.shape =",xs.shape)

    it = 0
    iTone = 0
    inds = ntrans[it] ==  ntranByTone[iTone]
    temp = xs[it, inds, streamByTone[iTone]][1000:]
    
    plt.plot(np.real(temp), label="I")
    plt.plot(np.imag(temp), label="Q")
    amplitude = np.abs(temp).mean()
    fi = np.angle(temp).mean()
    print(amplitude,fi)
    plt.title("freq=%.6f amp=%.3f  fi=%.6f"%(foutQuantized,amplitude,fi))
    return foutQuantized,amplitude, fi

In [9]:
# Create an output tone
def measurePhase(foutRequested, doPlot=False, verbose=False):
    soc.dds_out.alloff()
    foutQuantized = soc.DF*np.round(foutRequested/soc.DF)
    pfb_ch, dds_freq, _, _ = soc.pfb_out.freq2ch(foutQuantized, soc.get_mixer())
    soc.dds_out.ddscfg(ch=pfb_ch, f=dds_freq, g=0.9, fi=0.0)
    soc.pfb_out.qout(0)
    fInAliased = soc.fAliasedFromFTone(foutQuantized)
    decimation = 2
    pfbInQout = 8
    K, dds_freq, pdb_freq, ch = soc.pfb_in.freq2ch(np.array([fInAliased]))
    stream, stream_idx = soc.chsel.ch2idx(K)
    inCh, offset = soc.inFreq2chOffset(fInAliased)
    ntran, addr, bits = soc.chsel.ch2tran(inCh)
    num_tran, tran_idx = soc.chsel.set(K)
    ntranByTone, streamByTone = soc.inFreq2NtranStream(np.array([fInAliased]))

    soc.pfb_in.qout(pfbInQout)
    soc.ddscic.decimation(decimation)
    soc.ddscic.dds_outsel(outsel="product")
    soc.ddscic.set_ddsfreq(ch_id=inCh, f=offset)

    nt = 1
    nsamp = 10000
    packets = soc.stream.transfer(nt=nt, nsamp=nsamp)
    
    ntrans = packets[:,:,16]
    xis = packets[:,:,0:16:2]
    xqs = packets[:,:,1:17:2]
    xs = xis + 1j*xqs

    it = 0
    iTone = 0
    inds = ntrans[it] ==  ntranByTone[iTone]
    temp = xs[it, inds, streamByTone[iTone]][1000:]
    amplitude = np.abs(temp).mean()
    fi = np.angle(temp).mean()
    

    if verbose:
        print("output channel =",pfb_ch)
        print("output DDS freq =",dds_freq)
        print("foutQuantized =",foutQuantized)
        print("fInAliased =",fInAliased)
        print("K =",K, "dds_freq =",dds_freq)
        print("stream, stream_idx",stream,stream_idx)
        print("inCh =",inCh, "offset =",offset)
        print("num_tran, tran_idx",num_tran,tran_idx)
        print("ntranByTone, streamByTone",ntranByTone, streamByTone)
        print("     packets.shape =",packets.shape)
        print("      ntrans.shape =",ntrans.shape)
        print("         xis.shape =",xis.shape)
        print("         xqs.shape =",xqs.shape)
        print("          xs.shape =",xs.shape)

        print(amplitude,fi)

    if doPlot:
        plt.plot(np.real(temp), label="I")
        plt.plot(np.imag(temp), label="Q")
        plt.title("freq=%.6f amp=%.3f  fi=%.6f"%(foutQuantized,amplitude,fi))
    return foutQuantized,amplitude, fi

In [33]:
f0,a0,fi0 = measurePhase(foutRequested)

df = 0.001
f1,a1,fi1 = measurePhase(foutRequested+df)

delay = (fi1-fi0)/(f1-f0)
print("delay =",delay)

delay = 9.560199126411236
